# JUEGOS O BÚSQUEDA ADVERSARIAL

Este cuaderno sirve como material de apoyo para los temas tratados en el **Capítulo 5: Búsqueda adversaria** del libro *Inteligencia artificial: un enfoque moderno.* Este cuaderno utiliza implementaciones del módulo [games.py](https://github.com/aimacode/aima-python/blob/master/games.py). Importemos las clases, métodos, variables globales, etc. requeridos, desde el módulo de juegos.

# CONTENIDO

* Representación del juego
* Ejemplos de juegos
* Tres en raya
* Figura 5.2 Juego
* Mínimo máximo
* Alfa Beta
* Jugadores
* ¡Juguemos algunos juegos!

In [1]:
from games import *
from notebook import psource, pseudocode

# REPRESENTACIÓN DEL JUEGO

Para representar juegos utilizamos la clase `Game`, que podemos subclasificar y anular sus funciones para representar nuestros propios juegos. Una herramienta de ayuda es la tupla nombrada `GameState`, que en algunos casos puede resultar útil, especialmente cuando nuestro juego necesita que recordemos un tablero (como el ajedrez).

## `GameState` llamado tupla

`GameState` es un [namedtuple](https://docs.python.org/3.5/library/collections.html#collections.namedtuple) que representa el estado actual de un juego. Se utiliza para ayudar a representar juegos cuyos estados no se pueden representar fácilmente normalmente, o para juegos que requieren memoria de un tablero, como Tic-Tac-Toe.

`Gamestate` se define de la siguiente manera:

`GameState = nametuple('GameState', 'to_move, utilidad, tablero, movimientos')`

* `to_move`: Representa a quién le toca moverse a continuación.

* `utilidad`: Almacena la utilidad del estado del juego. Almacenar esta utilidad es una buena idea porque, cuando realiza una búsqueda Minimax o una búsqueda Alphabeta, genera muchas llamadas recursivas, que viajan hasta los estados terminales. Cuando estas llamadas recursivas regresan al destinatario original, hemos calculado utilidades para muchos estados del juego. Almacenamos estas utilidades en sus respectivos `GameState` para evitar calcularlas nuevamente.

* `tablero`: Un dictado que almacena el tablero del juego.

* `moves`: Almacena la lista de movimientos legales posibles desde la posición actual.

## Clase `Juego`

Echemos un vistazo a la clase "Juego" en nuestro módulo. Vemos que tiene funciones, a saber, `acciones`, `resultado`, `utilidad`, `terminal_test`, `to_move` y `display`.

Vemos que estas funciones en realidad no se han implementado. Esta clase es sólo una clase de plantilla; Se supone que debemos crear la clase para nuestro juego, heredando esta clase "Juego" e implementando todos los métodos mencionados en "Juego".

In [ ]:
%psource Game

Ahora entremos en detalles de todos los métodos de nuestra clase "Juego". Tienes que implementar estos métodos cuando creas nuevas clases que representen tu juego.

* `acciones(self, state)`: Dado un estado del juego, este método genera todas las acciones legales posibles a partir de este estado, como una lista o un generador. Devolver un generador en lugar de una lista tiene la ventaja de que ahorra espacio y aún puede operar con él como una lista.


* `resultado(self, state, move)`: dado un estado de juego y un movimiento, este método devuelve el estado de juego que se obtiene al realizar ese movimiento en este estado de juego.


* `utilidad(self, state, player)`: dado un estado de juego terminal y un jugador, este método devuelve la utilidad para ese jugador en el estado de juego terminal dado. Al implementar este método, suponga que el estado del juego es un estado de juego terminal. La lógica de este módulo es tal que este método sólo se llamará en estados de juego de terminal.


* `terminal_test(self, state)`: Dado un estado del juego, este método debería devolver `True` si este estado del juego es un estado terminal, y `False` en caso contrario.


* `to_move(self, state)`: dado el estado del juego, este método devuelve el jugador que jugará a continuación. Esta información normalmente se almacena en el estado del juego, por lo que todo lo que hace este método es extraer esta información y devolverla.


* `display(self, state)`: Este método imprime/muestra el estado actual del juego.

# EJEMPLOS DE JUEGOS

A continuación te damos algunos ejemplos de juegos que puedes crear y experimentar.

## Tres en raya

Eche un vistazo a la clase "TicTacToe". Todos los métodos mencionados en la clase "Juego" se han implementado aquí.

In [4]:
%psource TicTacToe

class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if 

La clase `TicTacToe` ha sido heredada de la clase `Game`. Como se mencionó anteriormente, realmente quieres hacer esto. Detectar errores y errores se vuelve mucho más fácil.

Métodos adicionales en TicTacToe:

* `__init__(self, h=3, v=3, k=3)` : Cuando creas una clase heredada de la clase `Game` (clase `TicTacToe` en nuestro caso), tendrás que crear un objeto de esta clase heredada para inicializar el juego. Esta inicialización podría requerir información adicional que se pasaría a `__init__` como variables. Para el caso de nuestro juego `TicTacToe`, esta información adicional sería el número de filas `h`, el número de columnas `v` y cuántas X u O consecutivas se necesitan en una fila, columna o diagonal para ganar `k `. Además, el estado inicial del juego debe definirse aquí en `__init__`.


* `compute_utility(self, board, move, player)`: un método para calcular la utilidad del juego TicTacToe. Si 'X' gana con este movimiento, este método devuelve 1; si 'O' gana, devuelve -1; de lo contrario devuelve 0.


* `k_in_row(self, board, move, player, delta_x_y)`: este método devuelve `True` si hay una línea formada en el tablero de TicTacToe con el último movimiento; de lo contrario, `False.`

### Estado del juego TicTacToe

Ahora, antes de comenzar a implementar nuestro juego "TicTacToe", debemos decidir cómo representaremos el estado de nuestro juego. Normalmente, el estado de un juego te brindará toda la información actual sobre el juego en cualquier momento. Cuando se te da un estado de juego, deberías poder saber quién es el siguiente turno, cómo se verá el juego en un tablero de la vida real (si lo tiene), etc. No es necesario que un estado de juego incluya la historia del juego. Si puedes seguir jugando el juego dado un estado del juego, la representación del estado del juego es aceptable. Si bien es posible que nos guste incluir todo tipo de información en el estado de nuestro juego, no queremos incluir demasiada información en él. Modificar este estado del juego para generar uno nuevo sería una verdadera molestia entonces.

Ahora, en cuanto al estado de nuestro juego "TicTacToe", ¿sería suficiente almacenar solo las posiciones de todas las X y O para representar toda la información del juego en ese momento? Bueno, ¿nos dice a quién le toca el siguiente? Mirar las "X" y las "O" en el tablero y contarlas debería decirnos eso. Pero eso significaría computación adicional. Para evitar esto, también almacenaremos cuál es el siguiente movimiento en el estado del juego.

Piensa en lo que hemos hecho aquí. Hemos reducido el cálculo adicional al almacenar información adicional en un estado de juego. Ahora bien, es posible que esta información no sea absolutamente esencial para informarnos sobre el estado del juego, pero nos ahorra tiempo de cálculo adicional. Haremos más de esto más adelante.

Para almacenar los estados del juego se utilizará la tupla nombrada `GameState`.

* `to_move`: Una cadena de un solo carácter, ya sea 'X' u 'O'.

* `utilidad`: 1 por victoria, -1 por pérdida, 0 en caso contrario.

* `tablero`: Todas las posiciones de las X y las O en el tablero.

* `moves`: Todos los movimientos posibles desde el estado actual. Tenga en cuenta aquí que almacenar los movimientos como una lista, como se hace aquí, aumenta la complejidad espacial de la búsqueda Minimax de `O(m)` a `O(bm)`. Consulte la sección 5.2.1 del libro.

### Representando un movimiento en el juego TicTacToe

Ahora que hemos decidido cómo se representará el estado de nuestro juego, es hora de decidir cómo se representará nuestro movimiento. Resulta fácil usar este movimiento para modificar el estado actual del juego y generar uno nuevo.

Para nuestro juego "TicTacToe", simplemente representaremos un movimiento mediante una tupla, donde el primer y segundo elemento de la tupla representarán la fila y la columna, respectivamente, donde se realizará el siguiente movimiento. Si hacer una 'X' o una 'O' lo decidirá `to_move` en la tupla nombrada `GameState`.

## Juego Fig52

Para un ejemplo más trivial representaremos el juego en la **Figura 5.2** del libro.

<img src="images/fig_5_2.png" width="75%">

Los estados se representan con letras mayúsculas dentro de los triángulos (por ejemplo, "A") mientras que los movimientos son las etiquetas en los bordes entre estados (por ejemplo, "a1"). Los nodos terminales llevan valores de utilidad. Tenga en cuenta que los nodos terminales se denominan en este ejemplo 'B1', 'B2' y 'B2' para los nodos debajo de 'B', y así sucesivamente.

Modelaremos los movimientos, utilidades y estado inicial así:

In [4]:
moves = dict(A=dict(a1='B', a2='C', a3='D'),
                 B=dict(b1='B1', b2='B2', b3='B3'),
                 C=dict(c1='C1', c2='C2', c3='C3'),
                 D=dict(d1='D1', d2='D2', d3='D3'))
utils = dict(B1=3, B2=12, B3=8, C1=2, C2=4, C3=6, D1=14, D2=5, D3=2)
initial = 'A'

En "movimientos", tenemos un sistema de diccionario anidado. El diccionario externo tiene claves como estados y valores de los posibles movimientos desde ese estado (como diccionario). El diccionario interno de movimientos tiene claves que nombran los movimientos y valoran el siguiente estado después de que se completa el movimiento.

A continuación se muestra un ejemplo que muestra "movimientos". Queremos el siguiente estado después del movimiento 'a1' de 'A', que es 'B'. Un vistazo rápido a la imagen de arriba confirma que este es efectivamente el caso.

In [5]:
print(moves['A']['a1'])

B


Ahora veremos las funciones que necesitamos implementar. Primero necesitamos crear un objeto de la clase `Fig52Game`.

In [6]:
fig52 = Fig52Game()

`acciones`: Devuelve la lista de movimientos que uno puede realizar desde un estado determinado.

In [ ]:
psource(Fig52Game.actions)

In [8]:
print(fig52.actions('B'))

['b1', 'b2', 'b3']


`resultado`: Devuelve el siguiente estado después de realizar un movimiento específico.

In [ ]:
psource(Fig52Game.result)

In [10]:
print(fig52.result('A', 'a1'))

B


`utilidad`: Devuelve el valor del estado terminal de un jugador ('MAX' y 'MIN'). Tenga en cuenta que para 'MIN' el valor devuelto es el negativo de la utilidad.

In [ ]:
psource(Fig52Game.utility)

In [12]:
print(fig52.utility('B1', 'MAX'))
print(fig52.utility('B1', 'MIN'))

3
-3


`terminal_test`: Devuelve `Verdadero` si el estado dado es un estado terminal, `Falso` en caso contrario.

In [ ]:
psource(Fig52Game.terminal_test)

In [14]:
print(fig52.terminal_test('C3'))

True


`to_move`: Devuelve el jugador que se moverá en este estado.

In [ ]:
psource(Fig52Game.to_move)

In [16]:
print(fig52.to_move('A'))

MAX


En su conjunto la clase `Fig52` que hereda de la clase `Game` y anula sus funciones:

In [ ]:
psource(Fig52Game)

# MÍNIMO MÁXIMO

## Descripción general

Este algoritmo (a menudo llamado *Minimax*) calcula el siguiente movimiento de un jugador (MIN o MAX) en su estado actual. Calcula recursivamente el valor minimax de los estados sucesores, hasta que llega a las terminales (las hojas del árbol). Utilizando el valor de "utilidad" de los estados terminales, calcula los valores de los estados principales hasta que llega al nodo inicial (la raíz del árbol).

Vale la pena señalar que el algoritmo funciona primero en profundidad. El pseudocódigo se puede encontrar a continuación:

In [2]:
pseudocode("Minimax-Decision")

### AIMA3e
__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__Figure__ ?? An algorithm for calculating minimax decisions. It returns the action corresponding to the best possible move, that is, the move that leads to the outcome with the best utility, under the assumption that the opponent plays to minimize utility. The functions MAX\-VALUE and MIN\-VALUE go through the whole game tree, all the way to the leaves, to determine the backed\-up value of a state. The notation argmax <sub>_a_ &Element; _S_</sub> _f_(_a_) computes the element _a_ of set _S_ that has maximum value of _f_(_a_).

## Implementación

En la implementación estamos usando dos funciones, `max_value` y `min_value` para calcular el mejor movimiento para MAX y MIN respectivamente. Estas funciones interactúan en una recursión alterna; uno llama al otro hasta que se alcanza un estado terminal. Cuando la recursividad se detiene, nos quedan puntuaciones para cada movimiento. Devolvemos el máximo. A pesar de devolver el máximo, también funcionará para MIN, ya que para MIN los valores son negativos (por lo tanto, el orden de los valores se invierte, por lo que cuanto más alto, mejor también para MIN).

In [ ]:
psource(minimax_decision)

## Ejemplo

Ahora jugaremos al juego Fig52 usando este algoritmo. Eche un vistazo al Fig52Game desde arriba para seguirlo.

Es el turno de MAX de moverse, y está en el estado A. Puede moverse a B, C o D, usando los movimientos a1, a2 y a3 respectivamente. El objetivo de MAX es maximizar el valor final. Entonces, para tomar una decisión, MAX necesita conocer los valores en los nodos antes mencionados y elegir el mayor. Después de MAX, le toca jugar a MIN. Entonces MAX quiere saber cuáles serán los valores de B, C y D después de que MIN juegue.

El problema entonces es qué movimiento hará MIN en B, C y D. Los estados sucesores de todos estos nodos son estados terminales, por lo que MIN elegirá el valor más pequeño para cada nodo. Entonces, para B elegirá 3 (del movimiento b1), para C elegirá 2 (del movimiento c1) y para D volverá a elegir 2 (del movimiento d3).

Veamos esto en código:

In [19]:
print(minimax_decision('B', fig52))
print(minimax_decision('C', fig52))
print(minimax_decision('D', fig52))

b1
c1
d3


Ahora MAX sabe que los valores de B, C y D son 3, 2 y 2 (producidos por los movimientos anteriores de MIN). El mayor es 3, que obtendrá con la jugada a1. Este es entonces el movimiento que realizará MAX. Veamos el algoritmo en plena acción:

In [20]:
print(minimax_decision('A', fig52))

a1


## Visualización

A continuación tenemos una visualización de juego simple usando el algoritmo. Después de ejecutar el comando, haz clic en la celda para avanzar en el juego. Puede ingresar sus propios valores a través de una lista de 27 números enteros.

In [2]:
from notebook import Canvas_minimax
from random import randint

In [ ]:
minimax_viz = Canvas_minimax('minimax_viz', [randint(1, 50) for i in range(27)])

# ALFA BETA

## Descripción general

Si bien *Minimax* es fantástico para calcular un movimiento, puede resultar complicado cuando el número de estados del juego aumenta. El algoritmo necesita buscar todas las hojas del árbol, que aumentan exponencialmente a su profundidad.

Para Tic-Tac-Toe, donde la profundidad del árbol es 9 (después del noveno movimiento, el juego termina), ¡podemos tener como máximo 9! estados terminales (como máximo porque no todos los nodos terminales están en el último nivel del árbol; algunos están más arriba porque el juego terminó antes del noveno movimiento). Esto no es tan malo, pero para problemas más complejos como el ajedrez, tenemos más de $10^{40}$ nodos terminales. Desafortunadamente no hemos encontrado una manera de eliminar el exponente, pero sí hemos encontrado formas de aliviar la carga de trabajo.

Aquí examinamos *podar* el árbol del juego, lo que significa eliminar partes del mismo que no necesitamos examinar. El tipo particular de poda se llama *alfa-beta*, y la búsqueda en su totalidad se llama *búsqueda alfa-beta*.

Para mostrar qué partes del árbol no necesitamos buscar, veremos el ejemplo `Fig52Game`.

En el juego de ejemplo, necesitamos encontrar el mejor movimiento para el jugador MAX en el estado A, que es el valor máximo de los movimientos posibles de MIN en los estados sucesores.

`MÁX(A) = MÁX(MÍN(B), MÍN(C), MÍN(D))`

`MIN(B)` es el mínimo de 3, 12, 8, que es 3. Entonces la fórmula anterior se convierte en:

`MÁX(A) = MÁX(3, MÍN(C), MÍN(D))`

El siguiente movimiento que comprobaremos es c1, lo que conduce a un estado terminal con utilidad de 2. Antes de continuar buscando en el estado C, volvamos a nuestra fórmula con el nuevo valor:

`MAX(A) = MAX(3, MIN(2, c2, .... cN), MIN(D) )`

No sabemos cuántos movimientos permite el estado C, pero sabemos que el primero da como resultado un valor de 2. ¿Necesitamos seguir buscando en C? La respuesta es no. El valor que MIN seleccionará en C será como máximo 2. Dado que MAX ya tiene la opción de elegir algo mayor que eso, 3 de B, no necesita seguir buscando en C.

En *alfa-beta* utilizamos dos parámetros adicionales para cada estado/nodo, *a* y *b*, que describen los límites de los posibles movimientos. El parámetro *a* denota la mejor opción (valor más alto) para MAX a lo largo de esa ruta, mientras que *b* denota la mejor opción (valor más bajo) para MIN. A medida que avanzamos actualizamos *a* y *b* y podamos una rama de nodo cuando el valor del nodo es peor que el valor de *a* y *b* para MAX y MIN respectivamente.

En el ejemplo anterior, después de la búsqueda en el estado B, MAX tenía un valor *a* de 3. Entonces, al buscar en el nodo C encontramos un valor menor que ese, 2, dejamos de buscar en C.

Puedes leer el pseudocódigo a continuación:

In [3]:
pseudocode("Alpha-Beta-Search")

### AIMA3e
__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


---
__Figure__ ?? The alpha\-beta search algorithm. Notice that these routines are the same as the MINIMAX functions in Figure ??, except for the two lines in each of MIN\-VALUE and MAX\-VALUE that maintain _&alpha;_ and _&beta;_ (and the bookkeeping to pass these parameters along).

## Implementación

Al igual que *minimax*, volvemos a utilizar las funciones `max_value` y `min_value`, pero esta vez utilizamos los valores *a* y *b*, actualizándolos y deteniendo la llamada recursiva si terminamos en nodos con valores peores. que *a* y *b* (para MAX y MIN). El algoritmo encuentra el valor máximo y devuelve el movimiento que lo genera.

La implementación:

In [21]:
%psource alphabeta_search

## Ejemplo

Jugaremos al juego Fig52 con el algoritmo de búsqueda *alfa-beta*. Es el turno de MAX de jugar en el estado A.

In [22]:
print(alphabeta_search('A', fig52))

a1


El movimiento óptimo para MAX es a1, por las razones expuestas anteriormente. MIN elegirá el movimiento b1 para B, lo que dará como resultado un valor de 3, actualizando el valor *a* de MAX a 3. Luego, cuando encontremos en C un nodo de valor 2, dejaremos de buscar en ese subárbol ya que es menor que *a*. De D tenemos un valor de 2. Entonces, el mejor movimiento para MAX es el que da como resultado un valor de 3, que es a1.

A continuación vemos los mejores movimientos para MIN comenzando desde B, C y D respectivamente. Tenga en cuenta que el algoritmo en estos casos funciona de la misma manera que *minimax*, ya que todos los nodos debajo de los estados antes mencionados son terminales.

In [23]:
print(alphabeta_search('B', fig52))
print(alphabeta_search('C', fig52))
print(alphabeta_search('D', fig52))

b1
c1
d3


## Visualización

A continuación encontrarás la visualización del algoritmo alfa-beta para un juego sencillo. Haz clic en la celda después de ejecutar el comando para avanzar en el juego. Puede ingresar sus propios valores a través de una lista de 27 números enteros.

In [2]:
from notebook import Canvas_alphabeta
from random import randint

In [ ]:
alphabeta_viz = Canvas_alphabeta('alphabeta_viz', [randint(1, 50) for i in range(27)])

# JUGADORES

Entonces, hemos terminado la implementación de las clases `TicTacToe` y `Fig52Game`. Estas clases lo que hacen es definir las reglas de los juegos. Necesitamos más para crear una IA que realmente pueda jugar. Aquí es donde entran `random_player` y `alphabeta_player`.

## jugador_consulta
La función `query_player` te permite a ti, un oponente humano, jugar. Esta función requiere que se implemente un método "display" en su clase de juego, de modo que los estados sucesivos del juego puedan mostrarse en el terminal, lo que le facilitará visualizar el juego y jugar en consecuencia.

## jugador_aleatorio
El `random_player` es una función que realiza movimientos aleatorios en el juego. Eso es todo. No hay mucho más para este chico.

## alfabeto_player
El `alphabeta_player`, por otro lado, llama a la función `alphabeta_search`, que devuelve el mejor movimiento en el estado actual del juego. Por lo tanto, `alphabeta_player` siempre realiza el mejor movimiento dado el estado del juego, asumiendo que el árbol del juego es lo suficientemente pequeño como para realizar una búsqueda completa.

## minimax_player
El `minimax_player`, por otro lado, llama a la función `minimax_search` que devuelve el mejor movimiento en el estado actual del juego.

## jugar un juego
La función `play_game` será la que realmente se utilizará para jugar. Le pasas como argumentos una instancia del juego que quieres jugar y los jugadores que quieres en este juego. ¡Úselo para jugar partidas de IA contra IA, IA contra humanos o incluso partidos de humano contra humano!

# ¡JUEGUEMOS ALGUNOS JUEGOS!

##

Comencemos experimentando primero con `Fig52Game`. Para eso crearemos una instancia de la subclase Fig52Game heredada de la clase Game:

In [27]:
game52 = Fig52Game()

Primero probamos nuestro `random_player(game, state)`. Dado un estado del juego, nos dará un movimiento aleatorio cada vez:

In [28]:
print(random_player(game52, 'A'))
print(random_player(game52, 'A'))

a1
a3


El `alphabeta_player(game, state)` siempre nos dará el mejor movimiento posible, para el jugador relevante (MAX o MIN):

In [29]:
print( alphabeta_player(game52, 'A') )
print( alphabeta_player(game52, 'B') )
print( alphabeta_player(game52, 'C') )

a1
b1
c1


Lo que hace `alphabeta_player` es simplemente llamar al método `alphabeta_full_search`. Ambos son esencialmente iguales. En el módulo se han implementado tanto `alphabeta_full_search` como `minimax_decision`. Ambos hacen el mismo trabajo y devuelven lo mismo, que es el mejor movimiento en el estado actual. Es solo que `alphabeta_full_search` es más eficiente con respecto al tiempo porque poda el árbol de búsqueda y, por lo tanto, explora un menor número de estados.

In [30]:
minimax_decision('A', game52)

'a1'

In [31]:
alphabeta_search('A', game52)

'a1'

Demostrando la función play_game en game52:

In [32]:
game52.play_game(alphabeta_player, alphabeta_player)

B1


3

In [33]:
game52.play_game(alphabeta_player, random_player)

B2


12

In [34]:
game52.play_game(query_player, alphabeta_player)

current state:
A
available moves: ['a1', 'a2', 'a3']

Your move? a1
B1


3

In [35]:
game52.play_game(alphabeta_player, query_player)

current state:
B
available moves: ['b1', 'b2', 'b3']

Your move? b1
B1


3

Tenga en cuenta que si es el primer jugador, Alphabeta_player juega como MIN, y si es el segundo jugador, Alphabeta_player juega como MAX. Esto sucede porque así es como se define el juego en la clase Fig52Game. Echar un vistazo al código de esta clase debería aclararlo.

## tres en raya

Ahora juguemos al "TicTacToe". Primero inicializamos el juego creando una instancia de la subclase TicTacToe heredada de la clase Juego:

In [36]:
ttt = TicTacToe()

Podemos imprimir un estado usando el método de visualización:

In [37]:
ttt.display(ttt.initial)

. . . 
. . . 
. . . 


Hmm, ese es el estado inicial del juego; sin X ni O.

Creemos un nuevo estado de juego nosotros mismos para experimentar:

In [38]:
my_state = GameState(
    to_move = 'X',
    utility = '0',
    board = {(1,1): 'X', (1,2): 'O', (1,3): 'X',
             (2,1): 'O',             (2,3): 'O',
             (3,1): 'X',
            },
    moves = [(2,2), (3,2), (3,3)]
    )

Entonces, ¿cómo es el estado del juego?

In [39]:
ttt.display(my_state)

X O X 
O . O 
X . . 


El `random_player` se comportará como se supone que debe hacerlo, es decir, *pseudoaleatorio*:

In [40]:
random_player(ttt, my_state)

(2, 2)

In [41]:
random_player(ttt, my_state)

(2, 2)

Pero `alphabeta_player` siempre dará el mejor movimiento, como se esperaba:

In [42]:
alphabeta_player(ttt, my_state)

(2, 2)

Ahora hagamos que dos jugadores jueguen uno contra el otro. Usamos la función `play_game` para esto. La función `play_game` hace que los jugadores jueguen el partido entre sí y devuelve la utilidad para el primer jugador, del estado terminal alcanzado cuando finaliza el juego. Por lo tanto, para nuestro juego "TicTacToe", si obtenemos el resultado +1, el primer jugador gana, -1 si gana el segundo jugador y 0 si el partido termina en empate.

In [43]:
ttt.play_game(random_player, alphabeta_player)

O O . 
X O X 
X X O 


-1

La salida es (normalmente) -1, porque `random_player` pierde frente a `alphabeta_player`. A veces, sin embargo, `random_player` logra dibujar con `alphabeta_player`.

Dado que un `alphabeta_player` juega perfectamente, una partida entre dos `alphabeta_player`s siempre debería terminar en empate. A ver si pasa esto:

In [44]:
for _ in range(10):
    print(ttt.play_game(alphabeta_player, alphabeta_player))

X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0


Un `random_player` nunca debería ganar contra un `alphabeta_player`. Probemos eso.

In [45]:
for _ in range(10):
    print(ttt.play_game(random_player, alphabeta_player))

X O O 
X O . 
O X X 
-1
O X . 
O X X 
O . . 
-1
X X O 
O O X 
O X . 
-1
O O O 
. X X 
X . . 
-1
O O O 
. . X 
X . X 
-1
O X O 
X O X 
X . O 
-1
O X X 
O X X 
O O . 
-1
O O X 
X O X 
X O . 
-1
O O X 
X O . 
X O X 
-1
O O X 
X X O 
O X X 
0


## Canvas_TicTacToe(Lienzo)

Esta subclase se utiliza para jugar al juego TicTacToe de forma interactiva en cuadernos Jupyter. La clase TicTacToe se llama al inicializar esta subclase.

Hagamos una coincidencia entre `random_player` y `alphabeta_player`. Haga clic en el tablero para llamar a los jugadores a hacer un movimiento.

In [46]:
from notebook import Canvas_TicTacToe

In [47]:
bot_play = Canvas_TicTacToe('bot_play', 'random', 'alphabeta')

Ahora, juguemos nosotros mismos contra un `jugador_aleatorio`:

In [48]:
rand_play = Canvas_TicTacToe('rand_play', 'human', 'random')

¡Hurra! Nosotros (normalmente) ganamos. Pero no podemos ganarle a un `alphabeta_player`, por mucho que lo intentemos.

In [49]:
ab_play = Canvas_TicTacToe('ab_play', 'human', 'alphabeta')